# Projeto Mineração de dados - Partidas Ranqueadas do tier Diamante do League of Legends

## Classificar a vitória da partida ranqueada sobre dados provenientes dos 10 primeiros minutos de jogo

https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min

**League of Legends** é consideram um MOBA (multiplayer online battle arena), os jogadores assumem o papel de "invocadores", **controlando campeões com habilidades únicas e que lutam com seu time de cinco integrantes** contra outros invocadores ou campeões controlados pelo computador. No modo mais popular do jogo, o objetivo de cada time é destruir o nexus da equipe adversária, uma construção localizada na base e protegida por outras estruturas.**Cada partida de League of Legends é distinta**, pois os campeões sempre começam fracos e progridem através da acumulação de ouro e da experiência ao longo da partida.

* League of Legends foi bem recebido desde o seu **lançamento em 2009 e sua popularidade cresceu ao decorrer dos anos**. 

* **Em julho de 2012**, foi **o jogo para computador mais jogado na América do Norte e Europa em termos de número de horas jogadas**. 

* **Até janeiro de 2014**, mais de **67 milhões de pessoas jogavam League of Legends por mês**, **27 milhões por dia** e **mais de 7,5 milhões durante o horário de pico**.

Video abaixo é uma rápida introdução ao mundo competitivo de League of Legends e também explica como funciona o jogo:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('2HodOC4fRNQ')

**Competições regionais semelhantes existem na China, Coreia, Taiwan e Sudeste da Ásia, Brasil, América Latina, Turquia, CEI (Comunidade dos Estados Independentes - antiga União Soviética) e Japão**. Essas competições regionais levam os melhores times ao League of Legends World Championship, um campeonato mundial que ocorre anualmente. 

Em 2013, o prêmio do torneio foi de 1 milhão de dolares e teve 32 milhões de espectadores onlines,  o torneio de 2014 teve o **quinto maior prêmio da história de eSports**, premiando **2,3 milhões de dólares para o time vencedor**.

Video em seguida detalha melhor o cenário mundial do competivo de League of Legends e também o tão aguardado World 2020.

In [ ]:
YouTubeVideo('dyrV0vicmBk')

## Import das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, plot_roc_curve, roc_curve,classification_report,confusion_matrix

In [ ]:
def quartils(df,coluna):
    print('Quartil de .90 '+str(df[coluna].quantile(0.90)))
    print('3° Quartil '+str(df[coluna].quantile(0.75)))
    print('2° Quartil '+str(df[coluna].quantile(0.50)))
    print('1° Quartil '+str(df[coluna].quantile(0.25))) 
    print(np.std(df[coluna]))

In [ ]:
def cv_score_metricas(score):
    print('Média : '+str(np.mean(score)))
    print('Desvio Padrão : '+str(np.std(score)))
    print('Minimo : '+str(np.min(score)))
    print('Maximo : '+str(np.max(score)))

## Read dos Dados

A base de dados foi extraida do Kaggle:

https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min

Provavelmente por mineração de dados de estatísticas de partidas reais realizadas no jogo

O objetivo com esse dataset é prever qual time vai ganhar em uma partida ranqueada de League Of Legends um dos tiers mais altos do jogo, Diamante e Mestre.

In [ ]:
df = pd.read_csv('../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv')

### Dicionário dos Dados

* **gameId** :                         Código de Identificação da partida
* **blueWins** :                        Atributo Binário para indicar se o time azul ganhou (**Target do dataset**)
* **blueWardsPlaced** :                 Wards colocadas em campo pelo time azul
* **blueWardsDestroyed** :              Wards Vermelhas destruidas
* **blueFirstBlood** :                  Atributo Binário se o time azul realizou a primeira eliminação da partida
* **blueKills** :                       Número de abates do time azul
* **blueDeaths** :                      Número de abates sofridos do time azul
* **blueAssists** :                     Número de assistencias em abates do time azul
* **blueEliteMonsters** :               Quantidade de monstros especiais abatidos pelo time azul
* **blueDragons** :                     Quantidade de dragões abatidos pelo time azul
* **blueHeralds** :                   Quantidade de Arautos abatidos pelo time azul
* **blueTowersDestroyed** :             Quantidade de Torres ou estruturas inimigas destruidas pelo time azul
* **blueTotalGold** :                   Quantidade total de gold coletada pelo time azul
* **blueAvgLevel** :                    Média de nível alcançado dos jogadores do time azul
* **blueTotalExperience** :             Quantidade total de experiência obtida pelo time azul
* **blueTotalMinionsKilled** :          Quantidade total de tropas inimigas abatidas pelo time azul
* **blueTotalJungleMinionsKilled** :    Quantidade total de monstros da selva abatidas pelo time azul
* **blueGoldDiff** :                    Diferença de gold entre o time azul e vermelho
* **blueExperienceDiff** :              Diferença de expriência entre o time azul e vermelho
* **blueCSPerMin** :                    Tropas abatidas por minuto do time azul
* **blueGoldPerMin** :                  Gold por minuto do time azul
* **redWardsPlaced** :                  Wards colocadas em campo pelo time azul
* **redWardsDestroyed** :               Wards Azuis destruidas
* **redFirstBlood** :                   Atributo Binário se o time vermelho realizou a primeira eliminação da partida
* **redKills** :                        Número de abates do time vermelho
* **redDeaths** :                       Número de abates sofridos do time vermelho
* **redAssists** :                      Número de assistencias em abates do time vermelho
* **redEliteMonsters** :                Quantidade de monstros especiais abatidos pelo time vermelho
* **redDragons** :                      Quantidade de dragões abatidos pelo time vermelho
* **redHeralds** :                      Quantidade de Arautos abatidos pelo time vermelho
* **redTowersDestroyed** :              Quantidade de Torres ou estruturas inimigas destruidas pelo time vermelho
* **redTotalGold** :                    Quantidade total de gold coletada pelo time vermelho
* **redAvgLevel** :                     Média de nível alcançado dos jogadores do time vermelho
* **redTotalExperience** :              Quantidade total de experiência obtida pelo time vermelho
* **redTotalMinionsKilled** :           Quantidade total de tropas inimigas abatidas pelo time vermelho
* **redTotalJungleMinionsKilled** :     Quantidade total de monstros da selva abatidas pelo time vermelho
* **redGoldDiff** :                     Diferença de gold entre o time vermelho e o azul
* **redExperienceDiff** :               Diferença de experiência entre o time vermelho e o azul
* **redCSPerMin** :                     Tropas abatidas por minuto do time vermelho
* **redGoldPerMin** :                   Gold por minuto do time vermelho

## Análise dos Dados

In [ ]:
df.head(10)

In [ ]:
df.iloc[0]

In [ ]:
df['blueDragons'].describe()

In [ ]:
df.info()

Notamos que **não há presença de valores nulos** no dataset, todos as colunas são numericas do tipo **Int** e **Float** e vemos que as colunas são quase "duplicadas", pois possuem as mesmas informações relativas ao time Azul e Vermelho.

Portanto por experiencia própria no jogo, o "nascimento" de Dragons ocorre no minuto 5:00 e o Heralds no 8:00, o respawn dos mesmos após abatidos ocorre respectivamente nos 6:00 e 5:00, portanto apenas 1 dos times pode conquistar um desses obejtivos até o período de 10 mintuos estabelecido do dataset.

Existe o campo **redFirstBlood** e **BlueFirstBlood**, eles indicam qual dos time obteve o primeiro abate de jogador da partida, portanto, só pode ser conquistada por um time por partida. Ter essa informação tanto sobre o time azul e vermelho é ambiguo, então vamos deletar a correspondente do time vermelho.

O campo gameID é o código unico da partida, não tem valor para predição.

In [ ]:
# Vamos tirar o gameID que é um PK não possui valor para o modelo
del df['gameId']
# Vamos retirar o redFirstBlood pois é ambíguo conforme as regras, a coluna blueFirstBlood já explicita
del df['redFirstBlood']

Em um momento de Feature Engennier podemos criar uma métrica bem utilizada que é KDA, a soma do numero de abates (Kill) e Assistências (Assist) dividido pelo numero de Mortes (Death). Conseguimos ver uma análise rápida com esse atributo, um valor superior que 0 nós diz que o jogador perfomou mais abates e assitências e subsequente menos mortes, no geral uma boa performance. O inverso dessa situação vemos um jogador que morreu mais e deu mais ouro para o inimigo. Vemos uma ocorrência de uma Bola de Neve nesse index.

In [ ]:
df['blue_KDA'] = round((df.blueKills+df.blueAssists)/np.maximum(1,df.blueDeaths), 3)
df['red_KDA'] = round((df.redKills+df.redAssists)/np.maximum(1,df.redDeaths), 3)

Procuramos duplicatas de registro, a premissa do League of Legends é que cada jogo é único e logo não teremos algo assim no dataset. Por precaução faremos uma procura por duplicadas

In [ ]:
df.drop_duplicates(keep=False,inplace=True) # Não possui registros duplicados

## Mapa de Correlação

Vamos procurar correlações entre as features, precisamente as perfeitas.

In [ ]:
corrMatrix = df.corr()
plt.figure(figsize=(16, 12))
sn.heatmap(corrMatrix, annot=False)
plt.show()

Identificamos correlações perfeitas (Index igual a 1 OU -1) entre colunas, isso pode prejudicar a performance do modelo pela ambiguidade das informações. Vamos retirar uma dessas colunas:

In [ ]:
lista_cor = pd.DataFrame(df.corr().unstack().sort_values().drop_duplicates())
lista_cor.columns = ['correlacao_index']
lista_cor[(lista_cor['correlacao_index'] > 0.9) | (lista_cor['correlacao_index'] < -0.9)]

In [ ]:
# Delete de colunas redundantes
del df['redExperienceDiff']
del df['redGoldDiff']
del df['redKills']
del df['redDeaths']
del df['blueKills']
del df['blueDeaths']
del df['redCSPerMin']
del df['blueCSPerMin']
del df['blueGoldPerMin']
del df['redGoldPerMin']
del df['redAvgLevel']
del df['blueAvgLevel']

In [ ]:
# novo tamanho do dataset em colunas
len(df.columns[+1:])

Acompanho o competitivo e jogo casualmente League of Legends desde de 2012, uma possível hipotese para achar uma heuristica para determinar o vencedor do jogo é acumulo de Ouro (Gold) pelas equipes. O jogo é motivado por habilidade e estratérgia de seus jogadores, executar com sucesso esses dois pontos em jogo permite a obteção de recompensa em Ouro e no final se resume a um jogo Econômico onde ouro compra itens, itens deixa o jogador mais forte e jogador forte deixa time perto da vitória.

Sabendo da informação anterior temos que ver quando essa hipotese é quebrada, ou seja, um time com vantágem de Ouro Perde, pois o que ganha realmente o jogo é a destruição do Nexus Inimigo. É importante saber se é comum a ocorrencia desse tipo de jogo no dataset analisado, vamos dar foco nos quartils para ver a distruibuição do acumulo de ouro da váriavel *TotalGold*.

In [ ]:
quartils(df,'redTotalGold')

In [ ]:
quartils(df,'blueTotalGold')

Observamos que times no quartil .90 possuem em aproximadamente de 3035 à 3081 de gold de vantagem ao adversário.

In [ ]:
# Filtro de registros que o time vermelho possui um TotalGold dentro do quartil .90 e mesmo assim perdeu
df_highGoldred_lost = df[(df['redTotalGold'] >= df['redTotalGold'].quantile(0.90)) & (df['blueWins'] == 1)]

In [ ]:
# Filtro de registros que o time azul possui um TotalGold dentro do quartil .90 e mesmo assim perdeu
df_highGoldblue_lost = df[(df['blueTotalGold'] >= df['blueTotalGold'].quantile(0.90)) & (df['blueWins'] == 0)]

Temos abaixo um total de registros:

In [ ]:
(len(df_highGoldblue_lost)+len(df_highGoldred_lost))

Que é equivalente em porcentágem:

In [ ]:
round(((len(df_highGoldblue_lost)+len(df_highGoldred_lost))/len(df))*100,3)

A probabilidade de que um time que estava com uma vantagem econômica expressiva de 3000 gold aos 10 mintuos iniciais e posteriormente perde a partida é de **3%**, portanto, uma baixa probabilidade e que pododemos encarar como **outliers**.

In [ ]:
# Atual tamanho do dataset normal
len(df)

In [ ]:
df.drop(df_highGoldblue_lost.index, inplace = True)
df.drop(df_highGoldred_lost.index, inplace = True)
df.reset_index(drop=True, inplace = True)

In [ ]:
# Atual tamanho do dataset pós drop de outliers
len(df)

Vamos fazer a mesma coisa com KDA, segue a mesma heuristica, onde KDA alto indica habilidade avançada do time e na hipótese que um time com KDA alto perde.

In [ ]:
quartils(df,'red_KDA')

In [ ]:
quartils(df,'blue_KDA')

Bastante aproximadas os quartis entre os times azul e vermelho

In [ ]:
# Filtro de registros que o time vermelho possui um KDA dentro do quartil .90 e mesmo assim perdeu
df_highKDAred_lost = df[(df['red_KDA'] >= df['red_KDA'].quantile(0.90)) & (df['blueWins'] == 1)]

In [ ]:
# Filtro de registros que o time azul possui um KDA dentro do quartil .90 e mesmo assim perdeu
df_highKDAblue_lost = df[(df['blue_KDA'] >= df['blue_KDA'].quantile(0.90)) & (df['blueWins'] == 0)]

Temos abaixo um total de registros:

In [ ]:
(len(df_highKDAblue_lost)+len(df_highKDAred_lost))

Que é equivalente em porcentágem:

In [ ]:
round(((len(df_highKDAblue_lost)+len(df_highKDAred_lost))/len(df))*100,3)

In [ ]:
# Atual tamanho do dataset normal
len(df)

In [ ]:
df.drop(df_highKDAred_lost.index, inplace = True)
df.drop(df_highKDAblue_lost.index, inplace = True)
df.reset_index(drop=True, inplace = True)

In [ ]:
# Atual tamanho do dataset pós drop de outliers
len(df)

**Reduzimos ao total 519 registros ao todo.** Ainda podemos trabalhar um com dataset com bastante registros.

## Pré-processamento

Durante testes empiricos, foi percebido que a dinamica dos jogos muda bastante quando temos valor de TotalGold dentro e antes do Terceiro quartil, isso ocorre quando temos jogos que vão para a fase de Late Game, ou seja, o jogo transpassa dos 25 min onde majoriamente os times estão ambos com bastante gold e itens e o determinante é a realização de objetivos e a execução de boas Team Fights, um outro fenonemo que acontesse é famoso termo "Over", onde basta uma falha de qualquer um dos times em uma partida equilibrada para haja a vitória.

Jogos que são decididos em Mid Game são aqueles que há uma vitória unanime, ou seja, um time abriu uma ampla vantagem no inicio e aproveitou isso e finalizou o jogo antes que o time adversário reagisse.

Resumindo vamos dividir o dataset em duas formas:
* Jogos antes do terceiro Quartil (Foco em Early e Middle Game) com *df_1*
* Jogos dentro e depois do Terceiro Quartil (Foco em Late Game) com *df_2*

## Normalização e Scaler

In [ ]:
df[df.columns[+1:]] = preprocessing.scale(df[df.columns[+1:]])

### Filtro entre os datasers Over e Under

In [ ]:
df_1 = df[(df['redTotalGold'] < df['redTotalGold'].quantile(0.85))  | (df['blueTotalGold'] < df['blueTotalGold'].quantile(0.75))]
df_2 = df[(df['redTotalGold'] > df['redTotalGold'].quantile(0.85))  | (df['blueTotalGold'] > df['blueTotalGold'].quantile(0.75))]

In [ ]:
train_under_3quantile, test_under_3quantile= train_test_split(df_1,test_size=0.20, random_state=42)
train_over_3quantile, test_over_3quantile= train_test_split(df_2,test_size=0.20, random_state=42)

## Dataset antes do 3° Quatil (df_2)

In [ ]:
y_train= train_under_3quantile[train_under_3quantile.columns[0]]
X_train = train_under_3quantile[train_under_3quantile.columns[+1:]]

y_test= test_under_3quantile[test_under_3quantile.columns[0]]
X_test = test_under_3quantile[test_under_3quantile.columns[+1:]]

In [ ]:
X_train.shape

In [ ]:
y_train.shape

## Treinamento do Modelo

In [ ]:
log_clf_under = LogisticRegression(random_state=42)
log_clf_under.fit(X_train, y_train)

## Validação do modelo

In [ ]:
resultado_under = log_clf_under.predict(X_test)

In [ ]:
print(classification_report(y_test, resultado_under))

In [ ]:
cm = confusion_matrix(y_test, resultado_under)
y_test_under = y_test

In [ ]:
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True,cmap='Blues', fmt='g')

* Positivo Verdadeiro : 728 (39,1%)
* Negativo Verdadeiro : 708 (38,0%)
* Falso Positivo : 194 (10,4%)
* Falso Negativo : 214 (11,5%)

Total da amostra: 1860

Tivemos um aproveitamento regular no foco dos Early e Mid games, Falsos Positivos e Falsos negativos ocorrem quase na mesma proporção.

### Curva de ROC

In [ ]:
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

plot_roc_curve(log_clf_under, X_test, y_test)
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
plt.show() 

### Cross Validation (CV)

In [ ]:
cv_score = cross_validate(log_clf_under, X_train, y_train, cv=10, scoring='roc_auc')

In [ ]:
cv_score_metricas(cv_score['test_score'])

## Dataset igual e depois 3° Quatil (df_2)

In [ ]:
y_train= train_over_3quantile[train_over_3quantile.columns[0]]
X_train = train_over_3quantile[train_over_3quantile.columns[+1:]]

y_test= test_over_3quantile[test_over_3quantile.columns[0]]
X_test = test_over_3quantile[test_over_3quantile.columns[+1:]]

### Split do Dataset

In [ ]:
X_train.shape

In [ ]:
y_train.shape

## Treinamento do Modelo

In [ ]:
log_clf_over = GradientBoostingClassifier(random_state=42)
log_clf_over.fit(X_train, y_train)

## Validação do modelo

In [ ]:
resultado_over = log_clf_over.predict(X_test)

In [ ]:
print(classification_report(y_test, resultado_over))

In [ ]:
cm = confusion_matrix(y_test, resultado_over)
y_test_over = y_test

In [ ]:
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True,cmap='Blues', fmt='g')

* Positivo Verdadeiro : 373 (51,7%)
* Negativo Verdadeiro : 252 (34,9%)
* Falso Positivo : 40 (5,5%)
* Falso Negativo : 56 (7,7%)

Total da amostra: 721

### Curva de ROC

In [ ]:
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

plot_roc_curve(log_clf_over, X_test, y_test)
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
plt.show() 

### Cross Validation (CV)

In [ ]:
cv_score = cross_validate(log_clf_over, X_train, y_train, cv=10, scoring='roc_auc')

In [ ]:
cv_score_metricas(cv_score['test_score'])

## Avalição conjunta dos dois modelos

In [ ]:
preds = np.concatenate((resultado_under, resultado_over), axis=0)

In [ ]:
y = np.concatenate((y_test_under, y_test_over), axis=0)

In [ ]:
print(classification_report(y, preds))

In [ ]:
cm = confusion_matrix(y, preds)

In [ ]:
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True,cmap='Blues', fmt='g')

* Positivo Verdadeiro : 1101 (54,2%)
* Negativo Verdadeiro : 960 (47,3%)
* Falso Positivo : 234 (11,5%)
* Falso Negativo : 270 (13,3%)

Total da amostra: 2028

## Conclusão

**Conseguimos uma base de 80% de acurrácia, precisão, Recall e F1 na base de teste**, um sucesso considerando **outros cadernos no Kaggle chegam até 76%**, portanto, **um avanço de 4%**.

* https://www.kaggle.com/allanbruno/predicting-wins-based-on-early-game-75-accuracy (aprox. 76%) 
* https://www.kaggle.com/neelkudu28/league-of-legends-win-prediction-using-pycaret  (aprox. 73%)
* https://www.kaggle.com/xiyuewang/lol-how-to-win                                   (aprox. 73%)

**Dividir o dataset e tirar outliers gerou uma melhora na predição da classe.**

**Interessante seria ampliar as variáveis do dataset**: quais personagems foram selecionados, itens comprados, o indice de MMR dos jogadores e entre outros que poderiam agregar mais detalhe do dataset para melhorar na modelagem da inferência. 

Foi um desafio muito interessante de fazer, olhar um o jogo que jogo regularmente com outro ponto de vista e como isso aproveitar fatos e insights com certeza que agregou na minha gameplay.